In [5]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from math import isclose
from random import randint

In [6]:
timeDiff = 1e-1
timeEnd = 10

In [7]:
class CosmicBody:
    array = []
    shouldDestroyed = False
    
    def __init__(self, mass: float, speed: np.ndarray, position: np.ndarray):
        self.mass = mass
        self.speed = speed
        self.position = position
        self.route = [position]
        
    def move(self, timeDiff: float):
        '''Просчитывает положение тела через время timeDiff'''
        # Расчёт гравитационных взаимодействий
        deltaSpeed = np.zeros(3)
        
        for body in CosmicBody.array:
            # На самого себя сила тяготения не действует
            if body == self:
                continue
                
            # Расчёт изменения скорости от силы тяготения
            r = body.position - self.position
            
            rLength = (r[0] ** 2 + r[1] ** 2 + r[2] ** 2) ** 0.5
            deltaSpeed = deltaSpeed + r * body.mass / rLength ** 3 * timeDiff
            
        deltaSpeed = deltaSpeed * 6.6743015 / (10 ** 11)
        
        # Изменение положения в пространстве
        self.position = self.position + self.speed * timeDiff + deltaSpeed * (timeDiff ** 2) / 2
        
        # Запись новой позиции в путь для построения на графике
        self.route.append(self.position)
        
        self.collision()
        self.destroy()
        
    def draw(self, ax):
        '''Отрисовывает пути и точки пройденные телом'''
        # Преобразование массива векторов к массивам с координатами для matplotlib
        xs = []
        ys = []
        zs = []
        for point in self.route:
            xs.append(point[0])
            ys.append(point[1])
            zs.append(point[2])
        
        ax.plot(xs, ys, zs)
        ax.scatter(xs, ys, zs)
        
    def collision(self):
        '''Обрабатывает столкновения тел'''
        for body in CosmicBody.array:
            
            if body == self:
                continue
            
            a = self.route[-1]
            b = self.route[-2] if len(self.route) >= 2 else a
            
            # Если объект прошёл через другой - значит последние точки первого лежат
            # слева и справа от координат второго
            
            # Расчёт левой и правой точек
            xLeft = min(a[0], b[0])
            xRight = max(a[0], b[0])
            yLeft = min(a[1], b[1])
            yRight = max(a[1], b[1])
            zLeft = min(a[2], b[2])
            zRight = max(a[2], b[2])
            
            #Проверка на прохождение через объект
            if xLeft <= body.position[0] <= xRight and \
            yLeft <= body.position[1] <= yRight and \
            zLeft <= body.position[2] <= zRight:
                # Уничтожается объект с наименьшей массой
                if self.mass < body.mass:
                    self.route[-1] = body.position
                    self.shouldDestroyed = True
                else:
                    body.shouldDestroyed = True
                
    def destroy(self):
        '''Уничтожает объект если shouldDestroyed = True'''
        if(self.shouldDestroyed):
            CosmicBody.array.remove(self)
            

In [8]:
class Star(CosmicBody):
    '''Звезда - такое же космическое тело, только нулевой скоростью и нулевыми координатами'''
    def __init__(self, mass: float):
        super().__init__(mass, np.zeros((3)), np.zeros((3)))